In [57]:
import json
file0 = r'C:\GitHub\IDFHIR JSON Payload Builder\IDFHIR-JSON-Builder\Postman collection manipulator\02. SATUSEHAT - Modul Pelayanan Rawat Jalan.postman_collection (1).json'
with open(file0, 'r') as file:
    dict_collection = json.load(file)

dict_collection

{'info': {'_postman_id': '7314bbda-26d3-4317-8af1-25171f06b05f',
  'name': '02. SATUSEHAT - Modul Pelayanan Rawat Jalan',
  'description': 'For **complete repository for all SATUSEHAT use-cases** please access our public workspaces : [Postman Workspaces](https://s.id/PostmanSATUSEHAT)\n\nFor **detailed SATUSEHAT use-cases import link** please see : [Postman Public Kemkes Link](https://link.kemkes.go.id/PostmanSATUSEHAT)\n\nAny information about SATUSEHAT FHIR Profile could be accessed through [Simplifier FHIR Profile](https://simplifier.net/guide/SATUSEHAT-FHIR-R4-Implementation-Guide/Home/FHIRProfiles?version=current)\n\nFor this Folder 02. Pelayanan Rawat Jalan, there are several parameterized variables, please kindly check folder-variables as well as tests on POST (Encounter, ServiceRequest, Specimen) As well as GET (Patient, Practitioner, Location, EpisodeOfCare)\n\nPlease make sure that you have inserted Org_id in your environment before testing\n\n| **Folder History** | **Editor*

In [82]:
import os
import json
import re

def sanitize_filename(filename, replace_space=' '):
    """
    Sanitize a string to make it safe for use as a filename or directory name.
    
    Args:
    - filename (str): The original filename string to be sanitized.
    - replace_space (str): Character to replace spaces with. Default is '_'.
    
    Returns:
    - str: A sanitized string safe for use as a filename or directory name.
    """
    # Replace spaces with a specific character or underscore
    filename = re.sub(r'\s+', replace_space, filename)

    # Remove or replace invalid filename characters
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)

    # Optional: Remove any non-ASCII characters or use unidecode for a better approach
    # filename = re.sub(r'[^\x00-\x7F]+', '', filename)  # Uncomment to remove non-ASCII
    # from unidecode import unidecode
    # filename = unidecode(filename)  # Uncomment if using unidecode

    return filename

# Example usage
original_filename = "example: filename<>with|special?*characters/"
safe_filename = sanitize_filename(original_filename)
print("Original:", original_filename)
print("Sanitized:", safe_filename)


def build_and_save_file(item, current_path=None):
    if current_path is None:
        current_path = []  # Initialize the path list if it's the first call

    # Check if the input is a list and iterate over each element
    if isinstance(item, list):
        for subitem in item:
            build_and_save_file(subitem, current_path)
    elif isinstance(item, dict):
        # Process the dictionary
        name = item.get('name')
        if name:
            # Append the current item's name to the path list
            path= sanitize_filename(name)
            print('current path= ', current_path)
            new_path = current_path + [path]
            print('new path= ', new_path)

        # Check if the request is POST and if raw data exists
            if ('request' in item and 'body' in item['request'] and 'raw' in item['request']['body'] 
                and item['request'].get('method') == "POST"):
                # Build the file path from the current path list
                base_path= r'C:\\Result'
                file_direct= os.path.join(base_path, *current_path)
                print('file_directory= ',file_direct)
                
                # Ensure the directory exists
                os.makedirs(file_direct, exist_ok=True)
                
                file_path = os.path.join(base_path, *new_path)
                final_path = os.path.join(file_path + '.json')
                print('final_path= ', final_path)

                # Assuming item['request']['body']['raw'] is a string containing JSON
                try:
                    # Parse the raw JSON string into a Python dictionary
                    raw_data = json.loads(item['request']['body']['raw'])
                    # Save the JSON file
                    with open(final_path, 'w', encoding='utf-8') as file:
                        json.dump(raw_data, file, indent=4)
                        print(f'File saved at: {final_path}')
                except json.JSONDecodeError as e:
                    print(f"Failed to decode JSON from raw data: {e}")
            else:
                print(f"Skipping {name}: Not a POST request or no raw body.")

                # Recursively process the next item or items if 'item' key is present
            if 'item' in item:
                build_and_save_file(item['item'], new_path)

Original: example: filename<>with|special?*characters/
Sanitized: example filenamewithspecialcharacters


In [83]:

# Example usage with a properly formatted dictionary
collection_json = {
    'name': "root",
    'item': [
        {
            'name': "child1",
            'request': {
                'method': 'POST',
                'body': {
                    'raw': '\r \n{"key": "value1"}'
                }
            },
            'item': {
                'name': "child2",
                'request': {
                    'method': 'GET',
                    'body': {
                        'raw': '\r \n{"key": "value2"}'
                    },
                'item': [
                    {
                        'name': 'child2-1',
                        'request': {
                            'method': 'POST',
                            'body': {
                                'raw': '\r \n{"key": "value2"}'
                            }
                        }
                    }
                ]
                },
                'item': [
                    {
                        'name': "child3",
                        'request': {
                            'method': 'POST',
                            'body': {
                                'raw': '\r \n{"key": "value3"}'
                            }
                        }, 
                    }
                ]
            }
        }
    ]
}

# Start the file generation process
build_and_save_file(collection_json)

current path=  []
new path=  ['root']
Skipping root: Not a POST request or no raw body.
current path=  ['root']
new path=  ['root', 'child1']
file_directory=  C:\\Result\root
final_path=  C:\\Result\root\child1.json
File saved at: C:\\Result\root\child1.json
current path=  ['root', 'child1']
new path=  ['root', 'child1', 'child2']
Skipping child2: Not a POST request or no raw body.
current path=  ['root', 'child1', 'child2']
new path=  ['root', 'child1', 'child2', 'child3']
file_directory=  C:\\Result\root\child1\child2
final_path=  C:\\Result\root\child1\child2\child3.json
File saved at: C:\\Result\root\child1\child2\child3.json


In [84]:
build_and_save_file(dict_collection['item'])

current path=  []
new path=  ['O-Auth2']
Skipping O-Auth2: Not a POST request or no raw body.
current path=  ['O-Auth2']
new path=  ['O-Auth2', 'Generate Token']
Skipping Generate Token: Not a POST request or no raw body.
current path=  []
new path=  ['00. Membuat Struktur Organisasi dan Lokasi']
Skipping 00. Membuat Struktur Organisasi dan Lokasi: Not a POST request or no raw body.
current path=  ['00. Membuat Struktur Organisasi dan Lokasi']
new path=  ['00. Membuat Struktur Organisasi dan Lokasi', 'Organization - Create UKP, Kefarmasian, Laboratorium']
file_directory=  C:\\Result\00. Membuat Struktur Organisasi dan Lokasi
final_path=  C:\\Result\00. Membuat Struktur Organisasi dan Lokasi\Organization - Create UKP, Kefarmasian, Laboratorium.json
File saved at: C:\\Result\00. Membuat Struktur Organisasi dan Lokasi\Organization - Create UKP, Kefarmasian, Laboratorium.json
current path=  ['00. Membuat Struktur Organisasi dan Lokasi']
new path=  ['00. Membuat Struktur Organisasi dan Loka

In [ ]:
build_and_save_file(dict_collection['item'])

current path=  []
new path=  ['O-Auth2']
Skipping O-Auth2: Not a POST request or no raw body.
current path=  ['O-Auth2']
new path=  ['O-Auth2', 'Generate Token']
Skipping Generate Token: Not a POST request or no raw body.
current path=  []
new path=  ['00. Membuat Struktur Organisasi dan Lokasi']
Skipping 00. Membuat Struktur Organisasi dan Lokasi: Not a POST request or no raw body.
current path=  ['00. Membuat Struktur Organisasi dan Lokasi']
new path=  ['00. Membuat Struktur Organisasi dan Lokasi', 'Organization - Create UKP, Kefarmasian, Laboratorium']
file_directory=  C:\\Result\00. Membuat Struktur Organisasi dan Lokasi
final_path=  C:\\Result\00. Membuat Struktur Organisasi dan Lokasi\Organization - Create UKP, Kefarmasian, Laboratorium.json
File saved at: C:\\Result\00. Membuat Struktur Organisasi dan Lokasi\Organization - Create UKP, Kefarmasian, Laboratorium.json
current path=  ['00. Membuat Struktur Organisasi dan Lokasi']
new path=  ['00. Membuat Struktur Organisasi dan Loka